In [25]:
import os

os.chdir("../")

import pickle
from typing import List, Tuple

import numpy as np
import pandas as pd
from fedot.core.pipelines.pipeline import Pipeline
from tqdm import tqdm

from meta_automl.data_preparation.dataset import CustomDataset, DatasetIDType
from meta_automl.data_preparation.datasets_loaders.custom_datasets_loader import \
    CustomDatasetsLoader
from meta_automl.data_preparation.model import Model
from meta_automl.data_preparation.models_loaders import \
    KnowledgeBaseModelsLoader

2023-08-19 14:58:28,526 - No config file found at C:\Users\chern\.openml\config, using default configuration.


In [26]:
PATH_TO_RESULT_CSV = r"C:\Users\chern\Downloads\Telegram Desktop\results.csv"
PATH_TO_CALCULATED_DATA = r"C:\Users\chern\python_projects\MetaFEDOT\collected_data.pickle"

In [27]:
df = pd.read_csv(PATH_TO_RESULT_CSV, index_col=0)

best_df_pred= df.groupby("task_id").apply(lambda x: x.iloc[x["y_pred"].argmax()][x.columns]).reset_index(drop=True)
best_df_true = df.groupby("task_id").apply(lambda x: x.iloc[x["y_true"].argmax()][x.columns]).reset_index(drop=True)

In [28]:
# # Uncomment if want to calculate. Otherwise, load data in the cell bellow.

# knowledge_base_directory = "data/knowledge_base_1"

# def dataset_from_id_without_data_loading(dataset_id: DatasetIDType) -> CustomDataset:
#     """ Creates the CustomDataset object without loading the data. Use if your don't need the models
#     to load the datasets data into memory, or if you have loaded the cache manually. """
#     return CustomDataset(dataset_id)

# datasets_loader_builder = lambda: CustomDatasetsLoader(dataset_from_id_func=dataset_from_id_without_data_loading)

# models_loader = KnowledgeBaseModelsLoader(knowledge_base_directory, datasets_loader = datasets_loader_builder())

# df_datasets = models_loader.parse_datasets("all", "classification")

# def _get_best_pipelines_unique_indexes(dataset_models: List[Model], fitness_coef=-1) -> List[int]:
#     temp_df = pd.DataFrame(columns=["predictor", "fitness"])
#     temp_df["predictor"] = [calc_pipeline_hash(x.predictor) for x in dataset_models]
#     temp_df["fitness"] = [fitness_coef * x.fitness.value for x in dataset_models]
#     # Select top-1 pipeline
#     best_pipelines_unique_indexes = temp_df.groupby('predictor')['fitness'].idxmax().to_list()
#     return best_pipelines_unique_indexes

# def calc_pipeline_hash(pl: Pipeline)-> Tuple[str]:
#     return tuple(str(pl.get_edges())+str(pl.nodes))

# dataset_ids = []
# dataset_names = []
# pipeline_id = 0
# models = []
# dict_pipelines = dict()

# for task_id in tqdm(df_datasets.index):
#     dataset = df_datasets.loc[task_id]
#     try:
#         dataset_id = int(dataset.dataset_id)
#     except ValueError:
#         dataset_id = dataset.dataset_id

#     dataset_ids.append(dataset_id)
#     dataset_names.append(dataset["dataset_name"])

#     dataset_models = models_loader.load(
#         dataset_ids=[dataset_id],
#         fitness_metric="fitness",
#     )
#     best_pipelines_unique_indexes = _get_best_pipelines_unique_indexes(dataset_models)
#     for index in best_pipelines_unique_indexes:
#         model = dataset_models[index]
        
#         pipeline_hash = calc_pipeline_hash(model.predictor)
#         if pipeline_hash not in dict_pipelines:
#             dict_pipelines[pipeline_hash] = pipeline_id
#             models.append(model)
#             pipeline_id += 1

# collected_data = [
#     dataset_ids,
#     dataset_names,
#     models,
# ]
# with open(PATH_TO_CALCULATED_DATA, "wb") as f:
#     pickle.dump(collected_data, f)

In [29]:
with open(PATH_TO_CALCULATED_DATA, "rb") as f:
    collected_data = pickle.load(f)

dataset_ids, dataset_names, models = collected_data

c:\Users\chern\.virtualenvs\MetaFEDOT-0JHEtfLt\lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
c:\Users\chern\.virtualenvs\MetaFEDOT-0JHEtfLt\lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The identity link alias is deprecated. Use Identity instead. The identity link alias will be removed after the 0.15.0 release.
  warnings.warn(
c:\Users\chern\.virtualenvs\MetaFEDOT-0JHEtfLt\lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The inverse_power link alias is deprecated. Use InversePower instead. The inverse_power link alias will be removed after the 0.15.0 release.
  warnings.warn(
c:\Users\chern\.virtualenvs\MetaFEDOT-0JHEtfLt\lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The inverse_squared link alias is deprecated. Use InverseSquared instead. The inverse

In [ ]:
records = []
for _, row in best_df_pred.iterrows():
    record = {
        "dataset_id": dataset_ids[int(row["task_id"])],
        "dataset_name": dataset_names[int(row["task_id"])],
        "fitness": models[int(row["pipe_id"])].metadata["fitness"],
        "roc_auc": models[int(row["pipe_id"])].metadata["roc_auc"],
        "logloss": models[int(row["pipe_id"])].metadata["logloss"],
        "model_str": models[int(row["pipe_id"])].predictor,
    }
    records.append(record)    
pd.DataFrame.from_records(records).to_csv("surrogate_testset_pred.csv", index=False)

In [ ]:
records = []
for _, row in best_df_true.iterrows():
    record = {
        "dataset_id": dataset_ids[int(row["task_id"])],
        "dataset_name": dataset_names[int(row["task_id"])],
        "fitness": models[int(row["pipe_id"])].metadata["fitness"],
        "roc_auc": models[int(row["pipe_id"])].metadata["roc_auc"],
        "logloss": models[int(row["pipe_id"])].metadata["logloss"],
        "model_str": models[int(row["pipe_id"])].predictor,
    }
    records.append(record)    
pd.DataFrame.from_records(records).to_csv("surrogate_testset_true.csv", index=False)